# Notebook 2 - SQL avec vraies bases de données
## Analyse e-commerce avec PostgreSQL en ligne




### 🎯 Objectifs pédagogiques
- Connecter Python à une vraie base de données PostgreSQL
- Écrire des requêtes SQL complexes sur des données réelles
- Implémenter des analyses RFM avec SQL
- Intégrer SQL et pandas pour des analyses avancées
- Gérer les connexions et la sécurité

### 🛍️ Contexte du projet
Vous analysez les données d'un vrai dataset e-commerce (Brazilian E-Commerce Public Dataset) hébergé sur une base PostgreSQL.

Objectif : créer une segmentation clientèle pour optimiser les campagnes marketing.


## Partie 1 : Connexion à la base de données réelle

### 🔧 Installation et configuration


# Installation des dépendances


```
pip install psycopg2-binary sqlalchemy pandas python-dotenv
```




In [1]:
import psycopg2
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
#from dotenv import load_dotenv

### 🌐 Base de données PostgreSQL gratuite (ElephantSQL)

**Option 1 : ElephantSQL (20MB gratuit)**
1. Créez un compte sur [elephantsql.com](https://www.elephantsql.com/)
2. Créez une instance "Tiny Turtle" (gratuite)
3. Récupérez vos credentials

**Option 2 : Supabase (500MB gratuit)**
1. Créez un compte sur [supabase.com](https://supabase.com/)
2. Créez un nouveau projet
3. Récupérez l'URL de connexion PostgreSQL

In [2]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
load_dotenv()


# Configuration de connexion (à adapter selon votre provider)
DATABASE_CONFIG = {
"host":os.getenv('host'),
"database":os.getenv('database'),
"user":os.getenv('user'),
"password":os.getenv('password'),
"port":5432,
}

# Création de l'engine SQLAlchemy
engine = create_engine(
    f"postgresql://{DATABASE_CONFIG['user']}:{DATABASE_CONFIG['password']}@"
    f"{DATABASE_CONFIG['host']}:{DATABASE_CONFIG['port']}/{DATABASE_CONFIG['database']}"
)


# Test de connection
def test_connection():
    try:
        df = pd.read_sql("SELECT version()", con=engine)
        print("Connexion réussie !")
        print("Version PostgreSQL :", df.iloc[0, 0])
    except Exception as e:
        print(f"Erreur de connexion : {e}")
        return False
    return True


In [3]:
test_connection()

Connexion réussie !
Version PostgreSQL : PostgreSQL 17.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 13.2.0, 64-bit


True


## Partie 2 : Import du dataset e-commerce

### 📊 Dataset Brazilian E-Commerce
Nous utilisons le célèbre dataset Olist (100k commandes réelles).

**Tables à créer :**
1. **customers** : customer_id, customer_city, customer_state
2. **orders** : order_id, customer_id, order_status, order_date, order_delivered_date
3. **order_items** : order_id, product_id, seller_id, price, freight_value
4. **products** : product_id, product_category, product_weight_g
5. **sellers** : seller_id, seller_city, seller_state

In [4]:
### 📥 Import des données via API

import requests
import zipfile
import io
import random

def import_des_donnees():
    # 1. Import CSV en DataFrame
    
    customers_csv = "data/olist_customers_dataset.csv"
    products_csv = "data/olist_products_dataset.csv"
    sellers_csv = "data/olist_sellers_dataset.csv"
    orders_csv = "data/olist_orders_dataset.csv"
    order_items_csv = "data/olist_order_items_dataset.csv"

    # Chargementd des fichiers csv
    df_customers = pd.read_csv(customers_csv)
    df_products = pd.read_csv(products_csv)
    df_sellers = pd.read_csv(sellers_csv)
    df_orders = pd.read_csv(orders_csv)
    df_order_items = pd.read_csv(order_items_csv)
    
    return df_customers, df_products, df_sellers, df_orders, df_order_items,

def clean_data(df_customers, df_products, df_sellers, df_orders, df_order_items):
    print(" Nettoyage des données")

    # Renommer certaines colonnes si besoin
    df_products = df_products.rename(columns={"product_category_name": "product_category"})
    df_orders = df_orders.rename(columns={
    "order_purchase_timestamp": "order_date",
    "order_delivered_customer_date": "order_delivered_date"
})

    # Colonnes utiles uniquement
    df_customers = df_customers[["customer_id", "customer_city", "customer_state"]]
    df_products = df_products[["product_id", "product_category", "product_weight_g"]]
    df_sellers = df_sellers[["seller_id", "seller_city", "seller_state"]]
    df_orders = df_orders[["order_id", "customer_id", "order_status", "order_date", "order_delivered_date"]]
    df_order_items = df_order_items[["order_id", "product_id", "seller_id", "price", "freight_value"]]



    # # Suppression des doublons
    # df_customers.drop_duplicates(subset=["customer_id"], inplace=False)
    # df_products.drop_duplicates(subset=["product_id"], inplace=False)
    # df_sellers.drop_duplicates(subset=["seller_id"], inplace=False)
    # df_orders.drop_duplicates(subset=["order_id"], inplace=False)
    # df_order_items.drop_duplicates(subset=["order_id", "product_id", "seller_id"], inplace=False)

    print("Données nettoyées.")
    return df_customers, df_products, df_sellers, df_orders, df_order_items,


# Appel du pipeline complet
df_c, df_p, df_s, df_o, df_oi = import_des_donnees()
df_c, df_p, df_s, df_o, df_oi = clean_data(df_c, df_p, df_s, df_o, df_oi)

# Génération de données e-commerce réalistes
def generate_ecommerce_data():
    # Charger les clients existants
    df_customers = pd.read_csv("data/olist_customers_dataset.csv")

    # Étape 1 : Extraire 50 villes brésiliennes uniques
    villes_uniques = df_customers["customer_city"].unique()
    villes_selectionnees = random.sample(list(villes_uniques), 50)

    print("Extrait de 10 villes sélectionnées :")
    print(villes_selectionnees[:10])

    # Étape 2 : Générer des clients fictifs (id + ville + état)
    clients_simulés = []
    for i in range(10000):  # 10 000 clients
        ville = random.choice(villes_selectionnees)
        sous_df = df_customers[df_customers["customer_city"] == ville]
        if not sous_df.empty:
            etat = sous_df["customer_state"].iloc[0]
            client = {
                "customer_id": f"SIMULATED-{i}",
                "customer_city": ville,  
                "customer_state": etat
            }
            clients_simulés.append(client)

    df_clients_simulés = pd.DataFrame(clients_simulés)
    print(df_clients_simulés.head())

    return df_clients_simulés

# Appel du pipeline complet
df_c = generate_ecommerce_data()


"""
    Génère des données e-commerce réalistes

    Étapes guidées :
    1. Sélectionnez 50 villes brésiliennes aléatoirement
    2. Créez des clients avec distribution réaliste
    3. Générez des commandes avec saisonnalité
    4. Ajoutez des produits avec catégories cohérentes
    5. Calculez des prix et frais de port basés sur la distance
    """

 Nettoyage des données
Données nettoyées.
Extrait de 10 villes sélectionnées :
['itaueira', 'bueno brandao', 'tres barras', 'maravilhas', 'coronel macedo', 'fervedouro', 'areiopolis', 'montezuma', 'senador guiomard', 'dias d avila']
   customer_id     customer_city customer_state
0  SIMULATED-0           jussara             GO
1  SIMULATED-1              jupi             PE
2  SIMULATED-2           jussara             GO
3  SIMULATED-3     bueno brandao             MG
4  SIMULATED-4  francisco badaro             MG


'\n    Génère des données e-commerce réalistes\n\n    Étapes guidées :\n    1. Sélectionnez 50 villes brésiliennes aléatoirement\n    2. Créez des clients avec distribution réaliste\n    3. Générez des commandes avec saisonnalité\n    4. Ajoutez des produits avec catégories cohérentes\n    5. Calculez des prix et frais de port basés sur la distance\n    '

In [5]:
# CREATION DES TABLES SQL

from sqlalchemy import create_engine, text
import pandas as pd
import os
from dotenv import load_dotenv

def create_tables () :
    load_dotenv()
    engine = create_engine(
        f"postgresql://{os.getenv('user')}:{os.getenv('password')}@{os.getenv('host')}:{os.getenv('port')}/{os.getenv('database')}")

    with engine.connect() as conn:
        print("Création des tables...")
        
        create_customers = """
        CREATE TABLE IF NOT EXISTS customers (
            customer_id VARCHAR(50) PRIMARY KEY,
            customer_city VARCHAR(100),
            customer_state VARCHAR(2)
        );
        """

        create_products = """
        CREATE TABLE IF NOT EXISTS products (
            product_id VARCHAR(50) PRIMARY KEY,
            product_category VARCHAR(100),
            product_weight_g NUMERIC
        );
        """

        create_sellers = """
        CREATE TABLE IF NOT EXISTS sellers (
            seller_id VARCHAR(50) PRIMARY KEY,
            seller_city VARCHAR(100),
            seller_state VARCHAR(2)
        );
        """

        create_orders = """
        CREATE TABLE IF NOT EXISTS orders (
            order_id VARCHAR(50) PRIMARY KEY,
            customer_id VARCHAR(50),
            order_status VARCHAR(20),
            order_date TIMESTAMP,
            order_delivered_date TIMESTAMP,
            FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
        );
        """

        create_order_items = """
        CREATE TABLE IF NOT EXISTS order_items (
            order_id VARCHAR(50),
            product_id VARCHAR(50),
            seller_id VARCHAR(50),
            price NUMERIC,
            freight_value NUMERIC,
            FOREIGN KEY (order_id) REFERENCES orders(order_id),
            FOREIGN KEY (product_id) REFERENCES products(product_id),
            FOREIGN KEY (seller_id) REFERENCES sellers(seller_id)
        );
        """

        # Exécution des requêtes SQL
        conn.execute(text(create_customers))
        conn.execute(text(create_sellers))
        conn.execute(text(create_products))
        conn.execute(text(create_orders))
        conn.execute(text(create_order_items))
    
        conn.commit()
        print("Tables créées avec succès.")


       

In [6]:
create_tables()

Création des tables...
Tables créées avec succès.


In [11]:
def export_to_postgres(df_customers, df_products, df_sellers, df_orders, df_order_items):
    """
    Envoie les DataFrames dans les tables PostgreSQL (créées au préalable)
    """
    print("Exportation des données vers PostgreSQL...")
    load_dotenv()

    engine = create_engine(
        f"postgresql://{os.getenv('user')}:{os.getenv('password')}@{os.getenv('host')}:{os.getenv('port')}/{os.getenv('database')}"
    )

    # Filtrer les commandes dont le customer_id existe dans la table customers
    existing_customers = set(df_c['customer_id'])
    df_o = df_o[df_o['customer_id'].isin(existing_customers)]


    # Insertion dans les tables créées 
    df_customers.to_sql("customers", engine, index=False, if_exists="append")
    df_products.to_sql("products", engine, index=False, if_exists="append")
    df_sellers.to_sql("sellers", engine, index=False, if_exists="append")
    df_orders.to_sql("orders", engine, index=False, if_exists="append")
    df_order_items.to_sql("order_items", engine, index=False, if_exists="append")

    print("Données insérées avec succès.")

    # Vérification
    df_export = pd.read_sql("SELECT * FROM products LIMIT 5;", engine)
    print("Aperçu de la table 'products' :")
    print(df_export)
    

## Partie 3 : Requêtes SQL avancées


### 🔍 Analyses SQL à implémenter

#### 1. Analyse RFM (Récence, Fréquence, Montant)
```sql
-- Votre défi : Calculer les métriques RFM pour chaque client
WITH customer_metrics AS (
    SELECT
        c.customer_id,
        c.customer_state,
        -- Récence : jours depuis dernier achat
        -- Fréquence : nombre de commandes
        -- Montant : total dépensé
        
        -- Complétez cette requête CTE
        
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_id, c.customer_state
)

-- Créez les segments RFM (Champions, Loyaux, À risque, etc.)
SELECT
    customer_id,
    customer_state,
    recency_score,
    frequency_score,
    monetary_score,
    CASE
        WHEN recency_score >= 4 AND frequency_score >= 4 THEN 'Champions'
        WHEN recency_score >= 3 AND frequency_score >= 3 THEN 'Loyal Customers'
        -- Ajoutez les autres segments
        ELSE 'Others'
    END as customer_segment
FROM customer_metrics;
```

In [ ]:
#### 2. Analyse géographique des ventes

def geographic_sales_analysis():
    """
    Analysez les performances par état/région

    Requêtes à écrire :
    1. Top 10 des états par CA
    2. Croissance MoM par région
    3. Taux de conversion par ville
    4. Distance moyenne vendeur-acheteur
    """

    query_top_states = """
    -- Votre requête SQL ici
    -- Utilisez des JOINs et GROUP BY
    -- Calculez le CA, nombre de commandes, panier moyen
    """

    return pd.read_sql(query_top_states, engine)

#### 3. Analyse temporelle et saisonnalité
```sql
-- Détectez les patterns saisonniers
SELECT
    EXTRACT(YEAR FROM order_date) as year,
    EXTRACT(MONTH FROM order_date) as month,
    EXTRACT(DOW FROM order_date) as day_of_week,
    COUNT(*) as order_count,
    SUM(price + freight_value) as total_revenue,
    AVG(price + freight_value) as avg_order_value
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE order_status = 'delivered'
GROUP BY ROLLUP(
    EXTRACT(YEAR FROM order_date),
    EXTRACT(MONTH FROM order_date),
    EXTRACT(DOW FROM order_date)
)
ORDER BY year, month, day_of_week;
```

---

## Partie 4 : Analyse prédictive avec SQL

### 🔮 Modèles simples en SQL

In [ ]:
#### 1. Prédiction de churn

def churn_prediction_sql():
    """
    Identifiez les clients à risque de churn

    Indicateurs :
    - Pas d'achat depuis X jours
    - Baisse de fréquence d'achat
    - Diminution du panier moyen
    - Changement de comportement géographique
    """

    churn_query = """
    WITH customer_activity AS (
        -- Calculez les métriques d'activité récente
        -- Comparez avec l'historique du client
        -- Scorez le risque de churn
    )

    SELECT
        customer_id,
        days_since_last_order,
        order_frequency_trend,
        monetary_trend,
        churn_risk_score,
        CASE
            WHEN churn_risk_score > 0.7 THEN 'High Risk'
            WHEN churn_risk_score > 0.4 THEN 'Medium Risk'
            ELSE 'Low Risk'
        END as churn_segment
    FROM customer_activity;
    """

    return pd.read_sql(churn_query, engine)


#### 2. Recommandations produits
```sql
-- Market Basket Analysis simplifié
WITH product_pairs AS (
    SELECT
        oi1.product_id as product_a,
        oi2.product_id as product_b,
        COUNT(*) as co_purchase_count
    FROM order_items oi1
    JOIN order_items oi2 ON oi1.order_id = oi2.order_id
    WHERE oi1.product_id != oi2.product_id
    GROUP BY oi1.product_id, oi2.product_id
    HAVING COUNT(*) >= 10  -- Seuil minimum
)

SELECT
    product_a,
    product_b,
    co_purchase_count,
    co_purchase_count::float / total_a.count as confidence
FROM product_pairs pp
JOIN (
    SELECT product_id, COUNT(*) as count
    FROM order_items
    GROUP BY product_id
) total_a ON pp.product_a = total_a.product_id
ORDER BY confidence DESC;
```

---

## Partie 5 : Intégration avec les APIs météo

### 🌤️ Croisement données météo/ventes
```python
def weather_sales_correlation():
    """
    Correlez vos données météo du Notebook 1 avec les ventes
    
    Hypothèses à tester :
    1. Les ventes de certaines catégories augmentent-elles avec la pluie ?
    2. Y a-t-il un impact de la température sur les achats ?
    3. Les livraisons sont-elles impactées par la météo ?
    """
    
    # Récupérez les données météo historiques pour les villes brésiliennes
    weather_query = """
    SELECT DISTINCT customer_city, customer_state
    FROM customers
    WHERE customer_state IN ('SP', 'RJ', 'MG', 'RS', 'SC')
    ORDER BY customer_city;
    """
    
    cities = pd.read_sql(weather_query, engine)
    
    # Intégrez avec l'API météo
    # Analysez les corrélations
    
    pass
```

### 📊 Dashboard géo-temporel
```python
def create_geotemporal_dashboard():
    """
    Créez un dashboard interactif combinant :
    - Carte des ventes par région
    - Évolution temporelle avec météo
    - Segments clients géolocalisés
    - Prédictions par zone géographique
    """
    pass
```

---
## 🏆 Livrables finaux

### 📈 Rapport d'analyse complet
1. **Segmentation RFM (Recency, Frenquency, Monetary) ** : 5-7 segments avec caractéristiques
2. **Analyse géographique**  : Performances par région + recommandations
3. **Prédictions churn** : Liste des clients à risque + actions
4. **Recommandations produits** : Top 10 des associations
5. **Impact météo** : Corrélations significatives identifiées

### 🚀 Pipeline automatisé
```python
def automated_analysis_pipeline():
    """
    Pipeline qui :
    1. Se connecte à la DB
    2. Exécute toutes les analyses
    3. Met à jour les segments clients
    4. Génère le rapport automatiquement
    5. Envoie des alertes si nécessaire
    """
    pass
```

---

## 🎓 Auto-évaluation

- [ ] **Connexion DB** : PostgreSQL fonctionnelle
- [ ] **Requêtes complexes** : JOINs, CTEs, fonctions analytiques
- [ ] **Gestion des erreurs** : Connexions robustes
- [ ] **Performance** : Requêtes optimisées avec index
- [ ] **Intégration** : SQL + Python + APIs
- [ ] **Insights actionables** : Recommandations business claires

### 🔗 Préparation au Notebook 3
Le prochain notebook portera sur NoSQL (MongoDB) avec des données de réseaux sociaux et d'IoT, en temps réel.

### 💡 Bases de données alternatives
- **PlanetScale** : MySQL serverless gratuit
- **MongoDB Atlas** : 512MB gratuit
- **FaunaDB** : Base multi-modèle gratuite
- **Hasura Cloud** : GraphQL + PostgreSQL